# -1) imports

## -1.1) librairie standard

In [1]:
import pandas            as pd
import importlib
import notebook_importer
import time
import os

## -1.2) librairie de datavisualisation

In [2]:
from bokeh.io        import push_notebook, show, output_notebook
from bokeh.layouts   import row
from bokeh.plotting  import figure
from bokeh.plotting  import figure, output_file, show, ColumnDataSource
from bokeh.models    import HoverTool
from bokeh.transform import factor_cmap
from bokeh.palettes  import Spectral6, inferno
from bokeh.embed     import file_html
output_notebook()

Loading BokehJS ...

## -1.3) libraire spécifique au projet

In [3]:
import notebook_importer

from Velo        import Velo
from Ville       import Ville
from Trajet      import Trajet
from Logger      import Logger
from Station     import Station
from Cycliste    import Casseur
from Cycliste    import Cycliste
from Position    import Position as Position
from Prestataire import Prestataire
from transform_timestamp_in_date import transform_timestamp_in_date

importing Jupyter notebook from Velo.ipynb
importing Jupyter notebook from Ville.ipynb
importing Jupyter notebook from Trajet.ipynb
importing Jupyter notebook from Logger.ipynb
importing Jupyter notebook from Station.ipynb
importing Jupyter notebook from Cycliste.ipynb
importing Jupyter notebook from Position.ipynb
importing Jupyter notebook from Prestataire.ipynb
importing Jupyter notebook from transform_timestamp_in_date.ipynb


# 0) création de fonctions

In [4]:
from datetime import datetime

In [5]:
%matplotlib inline

In [6]:
def draw(positions):
    import pandas as pd
    df = pd.DataFrame([ position.get_json() for position in positions])
    df.plot.scatter(x="longitude", y="lattitude")

# 2) analyse des résultats

In [7]:
df = pd.read_csv("./../data/Cyclistes/cycliste_1.csv")

In [ ]:
if False:
    df = pd.read_csv("./../data/Cyclistes/cycliste_1.csv")

    df.groupby("velo")["position"].agg(["max", "min"])
    last_position = False
    keep_rows = []
    for r in df.iterrows():
        new_position = r[1]["position"]
        if last_position != new_position:
            keep_rows.append(r[1])
            last_position = new_position

    df = pd.DataFrame(keep_rows)

    from transform_timestamp_in_date import transform_timestamp_in_date as to_date

    df["h"] = df.timestamp.map(lambda x: to_date(x))

In [ ]:
if True:
    unique_id_cycliste = list(set([x.id for x in paris.cyclistes]))
    unique_id_cycliste = [str(x) for x in unique_id_cycliste]

    import seaborn as sns

    palette   = inferno(len(unique_id_cycliste))
    #color_map = sns.palplot(sns.color_palette("hls", 8))
    color_map = factor_cmap('id_cycliste',palette = palette, factors =  unique_id_cycliste)

    positions             = paris.get_historique_positions_cycliste(jitter=False)[['longitude', 'lattitude', 'id_cycliste', 'time',  ]]
    positions.id_cycliste = positions.id_cycliste.map(str)
    positions["type"]     = "position"
    positions             = ColumnDataSource(positions)


    df = paris.get_historique_positions_cycliste(jitter=False)
    df.head()

In [ ]:
if True:
    homes                 = paris.get_cycliste_homes()
    homes["type"]         = "homes"
    homes                 = ColumnDataSource(homes)

    workplaces            = paris.get_cycliste_workplaces()
    workplaces["type"]    = "workplaces"
    workplaces            = ColumnDataSource(workplaces)


    stations            = paris.get_topology()
    stations["type"]    = "stations"
    stations            = ColumnDataSource(stations)

    # Create a ColumnDataSource from df: source

    # Create the figure: p
    title = "Ville de {width} par {length} avec {nb_cycliste} cyclistes".format(**locals())
    p = figure(title=title)
    p.square(  'longitudes', 'lattitude', source= stations   , color="pink", size=max(40/width, 5))
    p.diamond( 'longitudes', 'lattitude', source= workplaces , color="cyan", size=max(40/width, 5))
    p.asterisk('longitudes', 'lattitude', source= homes      , color="red" , size=max(40/width, 5))


    p.circle('longitude', 'lattitude', source= positions, fill_color=color_map)# Add circle glyphs to the figure p


    hover = HoverTool(tooltips=[("cyliste", "@id_cycliste"),
                                ("type"   , "@type"),
                                ("time"   , "@time")])
    p.add_tools(hover)
    show(p)


    from bokeh.resources import CDN

    html = file_html(p, CDN, "my plot")
    with open("exemple_simulation.html", "w") as f:
        f.writelines(html)    

In [ ]:
if True:
    with open("exemple_simulation.html", "w") as f:
            f.writelines(html)    

    if True:
        df = pd.concat([velo.flush_data() for velo in paris.prestataire.velos])

        velo_les_plus_utilises = df.velo_id.value_counts().head()
        velo_les_plus_utilises

        velo_a_tester = velo_les_plus_utilises.index[0]
        df[df.velo_id==velo_a_tester].groupby(["velo_id", "timestamp"])["performance"].max()

    import os
    if True:
        log_dir         = './../data/Stations'

        fichiers_dons   = [fichier for fichier in os.listdir(log_dir) if fichier.startswith('donne_velo_log_stations_'  )]
        fichiers_gain   = [fichier for fichier in os.listdir(log_dir) if fichier.startswith('recupere_velo_log_stations')]

        df_dons         = [] 
        df_gains        = [] 

        for fichier_don in fichiers_dons:
            try                   : df_dons.append( pd.read_csv(f'{log_dir}/{fichier_don}', sep=",")  )
            except Exception as e : pass

        for fichier_gain in fichiers_gain:
            try                   : df_gains.append( pd.read_csv(f'{log_dir}/{fichier_gain}', sep=",")  )
            except Exception as e : pass

        df_dons          = pd.concat(df_dons, axis=0)
        df_dons["type"]  = "don"
        df_gains         = pd.concat(df_gains, axis=0)
        df_gains["type"] = "gain"
    df_gains.shape

    df_dons_gains          = pd.concat([df_gains, df_dons], ignore_index=True)
    df_dons_gains          = df_dons_gains[df_dons_gains.timestamp.astype(str)!="timestamp"]
    df_dons_gains.columns  = [colonne.replace(".", "_").replace(" ", "") for colonne in df_dons_gains.columns]
    df_dons_gains["velo_number"]   = df_dons_gains.velo_id.map(lambda x: int(x[x.find("_")-len(x)+1:]))
    df_dons_gains["timestamp_int"] = df_dons_gains.timestamp.astype(int)
    df_dons_gains.head()

    parcours_velo_a_tester = df_dons_gains[df_dons_gains.velo_id==velo_a_tester].sort_values("timestamp", ascending=True)
    parcours_velo_a_tester.head()

In [ ]:
_ = transform_timestamp_in_date(120)

In [ ]:
_.minute